In [ ]:
## 데이터 전처리
#1) 데이터 불러오기
#2) 세부적 전처리 과정은 데이터 보고 결정
## AI
#1) x(훈련 데이터), t(타겟 데이터) 입력
#2) 함수들 (신경망, 손실함수, 최적화함수) 선언
#3) y = F(x) (신경망 함수 처리, 순전파)
#4) 손실함수 (y와 t 비교)
#5) 최적화 함수 (신경망 수정, 역전파)
## 검증

In [2]:
import sklearn.datasets as datasets #데이터 불러오기
import numpy #데이터 수정
import pandas #데이터프레임
from IPython.display import display #데이터프레임 보는 용도
import torch #신경망
import torch.nn as nn #신경망 함수들

In [19]:
## 데이터 전처리
#1) 데이터 불러오기
#https://scikit-learn.org/stable/api/sklearn.datasets.html
wine_data = datasets.load_wine(return_X_y=True, as_frame=True)
#print(wine_data) #데이터 튜플 형태인것 확인
df_x = wine_data[0]
df_t = wine_data[1]

#2) 세부적 전처리 과정은 데이터 보고 결정
#결측치 제거
df_x = df_x.drop(df_x.columns[df_x.isnull().sum(axis = 0) > 0], axis = 1) #결측치 존재 행 제거
df_x = df_x.drop(df_x[df_x.isnull().sum(axis = 1) > 0].index, axis = 0) #결측치 존재 열 제거
#스케일링 - 표준화
for col in df_x.columns :
    mean = df_x[col].mean()
    std = df_x[col].std()
    df_x[col] = (df_x[col] - mean) / std
display(df_x)

display(df_x) #데이터 표 형식으로 보이기
display(df_t)


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,1.514341,-0.560668,0.231400,-1.166303,1.908522,0.806722,1.031908,-0.657708,1.221438,0.251009,0.361158,1.842721,1.010159
1,0.245597,-0.498009,-0.825667,-2.483841,0.018094,0.567048,0.731565,-0.818411,-0.543189,-0.292496,0.404908,1.110317,0.962526
2,0.196325,0.021172,1.106214,-0.267982,0.088110,0.806722,1.212114,-0.497005,2.129959,0.268263,0.317409,0.786369,1.391224
3,1.686791,-0.345835,0.486554,-0.806975,0.928300,2.484437,1.462399,-0.979113,1.029251,1.182732,-0.426341,1.180741,2.328007
4,0.294868,0.227053,1.835226,0.450674,1.278379,0.806722,0.661485,0.226158,0.400275,-0.318377,0.361158,0.448336,-0.037767
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,0.873810,2.966176,0.304301,0.300954,-0.331985,-0.982841,-1.420891,1.270726,-0.927563,1.139596,-1.388840,-1.227742,-0.021890
174,0.491955,1.408636,0.413653,1.049555,0.158126,-0.791103,-1.280731,0.547563,-0.316058,0.967055,-1.126341,-1.481267,0.009866
175,0.331822,1.739837,-0.388260,0.151234,1.418411,-1.126646,-1.340800,0.547563,-0.420888,2.217979,-1.607590,-1.481267,0.279786
176,0.208643,0.227053,0.012696,0.151234,1.418411,-1.030776,-1.350811,1.351077,-0.228701,1.829761,-1.563840,-1.396759,0.295664


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,1.514341,-0.560668,0.231400,-1.166303,1.908522,0.806722,1.031908,-0.657708,1.221438,0.251009,0.361158,1.842721,1.010159
1,0.245597,-0.498009,-0.825667,-2.483841,0.018094,0.567048,0.731565,-0.818411,-0.543189,-0.292496,0.404908,1.110317,0.962526
2,0.196325,0.021172,1.106214,-0.267982,0.088110,0.806722,1.212114,-0.497005,2.129959,0.268263,0.317409,0.786369,1.391224
3,1.686791,-0.345835,0.486554,-0.806975,0.928300,2.484437,1.462399,-0.979113,1.029251,1.182732,-0.426341,1.180741,2.328007
4,0.294868,0.227053,1.835226,0.450674,1.278379,0.806722,0.661485,0.226158,0.400275,-0.318377,0.361158,0.448336,-0.037767
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,0.873810,2.966176,0.304301,0.300954,-0.331985,-0.982841,-1.420891,1.270726,-0.927563,1.139596,-1.388840,-1.227742,-0.021890
174,0.491955,1.408636,0.413653,1.049555,0.158126,-0.791103,-1.280731,0.547563,-0.316058,0.967055,-1.126341,-1.481267,0.009866
175,0.331822,1.739837,-0.388260,0.151234,1.418411,-1.126646,-1.340800,0.547563,-0.420888,2.217979,-1.607590,-1.481267,0.279786
176,0.208643,0.227053,0.012696,0.151234,1.418411,-1.030776,-1.350811,1.351077,-0.228701,1.829761,-1.563840,-1.396759,0.295664


0      0
1      0
2      0
3      0
4      0
      ..
173    2
174    2
175    2
176    2
177    2
Name: target, Length: 178, dtype: int32

In [22]:
# t값 갯수 구하기
t_max = df_t.max()
t_min = df_t.min()
print(f"t_max : {t_max}")
print(f"t_min : {t_min}")
print()



t_max : 2
t_min : 0



NameError: name 'tensor_x' is not defined

In [23]:
## AI
#1) x(훈련 데이터), t(타겟 데이터) 입력
#torch는 torch.tensor 형태의 데이터만 받음,
x = df_x.values.tolist() #list로 데이터 변경
t = df_t.values.tolist()
tensor_x = torch.tensor(x, dtype = torch.float) #parameter : 행렬(list or numpy.array), 데이터 형식
tensor_t = torch.tensor(t, dtype = torch.long)
#2) 함수들 (신경망, 손실함수, 최적화함수) 선언
#신경망 
#fx(tensor_x.shape[-1],8) -> gx(8,4) -> hx(4,t_max-t_min+1)
F = nn.Sequential(
    nn.Linear(tensor_x.shape[-1],8),#fx
    nn.Sigmoid(),
    nn.Linear(8,4),#gx
    nn.Sigmoid(),
    nn.Linear(4,t_max-t_min+1),#hx
    nn.Sigmoid(),
)
#손실함수
loss_function = nn.CrossEntropyLoss()
#최적화 함수
optimizer = torch.optim.SGD(F.parameters(), lr = 0.015)
epoch = 30

for e in range(epoch) :
    loss_sum = 0
    for b in range(tensor_x.shape[0]) :
#3) y = F(x) (신경망 함수 처리, 순전파)
        y = F(tensor_x[b])
#4) 손실함수 (y와 t 비교)
        loss = loss_function(y, tensor_t[b])
        loss_sum += loss
#5) 최적화 함수 (신경망 수정, 역전파)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"epoch {e+1} || loss {loss_sum / tensor_x.shape[0]}")

epoch 1 || loss 1.1022253036499023
epoch 2 || loss 1.09865140914917
epoch 3 || loss 1.095728874206543
epoch 4 || loss 1.0933434963226318
epoch 5 || loss 1.0913912057876587
epoch 6 || loss 1.0897787809371948
epoch 7 || loss 1.0884324312210083
epoch 8 || loss 1.0872905254364014
epoch 9 || loss 1.0863038301467896
epoch 10 || loss 1.0854336023330688
epoch 11 || loss 1.0846487283706665
epoch 12 || loss 1.0839250087738037
epoch 13 || loss 1.0832414627075195
epoch 14 || loss 1.0825837850570679
epoch 15 || loss 1.0819365978240967
epoch 16 || loss 1.0812891721725464
epoch 17 || loss 1.0806306600570679
epoch 18 || loss 1.0799518823623657
epoch 19 || loss 1.07924485206604
epoch 20 || loss 1.0785011053085327
epoch 21 || loss 1.077711582183838
epoch 22 || loss 1.0768684148788452
epoch 23 || loss 1.0759615898132324
epoch 24 || loss 1.0749821662902832
epoch 25 || loss 1.0739201307296753
epoch 26 || loss 1.0727626085281372
epoch 27 || loss 1.0714973211288452
epoch 28 || loss 1.0701119899749756
epoch 2

In [25]:
## 검증
# 검증 데이터가 따로 없으니 훈련데이터로 검증
cnt = 0
y = F(tensor_x)
for i in range(tensor_x.shape[0]) :
    if torch.argmax(y[i]) == t[i] :
        cnt += 1

print(f"possibility : {cnt / tensor_x.shape[0]}")

#훈련 데이터가 적어서 성능이 낮음

possibility : 0.398876404494382
